*Instituto Europeo de Posgrado* - Master en Inteligencia Artificial Aplicada\
*Asignatura*: Data Science

# Unidad 2 - Casos Práticos

En esta unidad hemos aprendido a manejar y transformar datos a través de Pandas, una de las librerías más utilizadas en Python para el análisis de datos. En estas dos activades vamos a poner en práctica lo aprendido con dos conjuntos de datos reales.

Todos los ficheros a los que se hagan referencia en los ejercicios se encuentran en la carpeta *datos*.

Antes de comenzar, vamos a importar la librería Pandas que utilizaremos en ambos ejercicios.

In [129]:
import pandas as pd

## Ejercicio 1: Movie Ratings

En este ejercicio vamos a trabajar con un conjunto de datos que contiene información sobre películas y valoraciones de usuarios. 

El fichero *movies.csv* contiene las siguientes columnas:
* **movieId**: identificador de la película.
* **title**: título de la película.
* **genres**: géneros de la película. Están separados por el carácter "|".

El fichero *ratings.csv* contiene las siguientes columnas:

* **userId**: identificador del usuario que ha valorado la película.
* **movieId**: identificador de la película.
* **rating**: valoración de la película por el usuario, en una escala de 0.5 a 5.
* **timestamp**: fecha y hora en la que se realizó la valoración. Está en formato UNIX, es decir, el número de segundos que han pasado desde el 1 de enero de 1970. Luego veremos cómo convertirlo a un formato más legible.

### 1.1. Carga de datos

Lee los ficheros de datos y almacénalos en dos DataFrames de Pandas. Llámalos `movies` y `ratings`. ¿Cuántas películas y valoraciones hay en cada DataFrame?

In [132]:
import pandas as pd

movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [133]:
num_movies = movies.shape[0]
num_ratings = ratings.shape[0]

print(f'Películas: {num_movies}')
print(f'Valoraciones: {num_ratings}')

Películas: 45447
Valoraciones: 500000


### 1.2. Union de DataFrames

Añade el título y los géneros de las películas a la tabla de valoraciones de tal forma que tengamos una única tabla con toda la información. Llámala `movie_ratings`. Las películas que no tengan valoraciones o ratings de películas que no conocemos no nos interesan 

In [135]:
movie_ratings = ratings.merge(movies, on = 'movieId', how = 'inner')

print(movie_ratings.head())

   userId  movieId  rating   timestamp                           title  \
0       1      110     1.0  1425941529               Braveheart (1995)   
1       1      147     4.5  1425942435  Basketball Diaries, The (1995)   
2       1      858     5.0  1425941523           Godfather, The (1972)   
3       1     1221     5.0  1425941546  Godfather: Part II, The (1974)   
4       1     1246     5.0  1425941556       Dead Poets Society (1989)   

             genres  
0  Action|Drama|War  
1             Drama  
2       Crime|Drama  
3       Crime|Drama  
4             Drama  


### 1.3. ¿Cuantas valoraciones ha recibido la película "Dark Knight, The (2008)"? ¿Y de 5 estrellas?

In [137]:
dark_knight_ratings = movie_ratings[movie_ratings['title'] == 'Dark Knight, The (2008)']

total_ratings = dark_knight_ratings.shape[0]

five_star_ratings = dark_knight_ratings[dark_knight_ratings['rating'] == 5.0].shape[0]

print(f'Total de valoraciones: {total_ratings}')
print(f'Valoraciones de 5 estrellas: {five_star_ratings}')

Total de valoraciones: 794
Valoraciones de 5 estrellas: 240


### 1.4. Calcula la valoracion media y el número de valoraciones de cada película.

Almacena el resultado en un nuevo DataFrame llamado `movie_stats`. La funcion `size` nos da el número de valoraciones, y `mean` la valoración media.

In [139]:
movie_stats = movie_ratings.groupby('title')['rating'].agg(['mean', 'size']).reset_index()

movie_stats.rename(columns={'mean': 'valoracion_media', 'size': 'num_valoraciones'}, inplace=True)

print(movie_stats.head())

                              title  valoracion_media  num_valoraciones
0  "Great Performances" Cats (1998)          2.666667                 3
1                    #Horror (2015)          2.500000                 1
2                $ (Dollars) (1971)          2.500000                 1
3                   $5 a Day (2008)          2.500000                 1
4                      $9.99 (2008)          3.500000                 2


### 1.5. ¿Qué película tiene la valoración media más alta? 

Para tener un resutltado más fiable filtra aquellas películas que tengan al menos 100 valoraciones y extrae el top 10 de películas con la valoración media más alta.

In [141]:
peliculas_filtradas = movie_stats[movie_stats['num_valoraciones'] >= 100]

top_10_peliculas = peliculas_filtradas.sort_values(by='valoracion_media', ascending=False).head(10)

print(top_10_peliculas)

                                             title  valoracion_media  \
11985             Shawshank Redemption, The (1994)          4.421611   
14167     Treasure of the Sierra Madre, The (1948)          4.371681   
3997                       Double Indemnity (1944)          4.366972   
5516                         Godfather, The (1972)          4.361326   
14520                   Usual Suspects, The (1995)          4.309783   
11883  Seven Samurai (Shichinin no samurai) (1954)          4.283784   
11036                           Rear Window (1954)          4.279012   
9963        One Flew Over the Cuckoo's Nest (1975)          4.274905   
13826                        Third Man, The (1949)          4.270992   
1665                         Big Sleep, The (1946)          4.268182   

       num_valoraciones  
11985              1837  
14167               113  
3997                109  
5516               1161  
14520              1196  
11883               259  
11036               405  

### 1.6. Extracción del año de la película

El año de la película se encuentra entre paréntesis en el título, añade una columna al DataFrame `movies` con el año de la película asumiendo que siempre se encuentra en la misma posición. Utiliza una función lambda para extraer el año del título. Una vez que lo hayas extraído, elimina el año del título.

In [143]:
import re

movies['año'] = movies['title'].apply(lambda x: re.search(r'\((\d{4})\)', x).group(1) if re.search(r'\((\d{4})\)', x) else None)

movies['title'] = movies['title'].apply(lambda x: re.sub(r'\s*\(\d{4}\)', '', x))

print(movies.head())

   movieId                        title  \
0        1                    Toy Story   
1        2                      Jumanji   
2        3             Grumpier Old Men   
3        4            Waiting to Exhale   
4        5  Father of the Bride Part II   

                                        genres   año  
0  Adventure|Animation|Children|Comedy|Fantasy  1995  
1                   Adventure|Children|Fantasy  1995  
2                               Comedy|Romance  1995  
3                         Comedy|Drama|Romance  1995  
4                                       Comedy  1995  


### 1.7. Filtro por género

¿Cómo podríamos filtrar las películas de un género concreto? Por ejemplo, ¿cómo podríamos obtener las películas de acción de 1993? Piensa que la columna `genres` es una cadena de texto y por tanto podemos utilizar el método `str.contains` para filtrar por género.

In [145]:
# Filtrar películas de acción del año 1993
peliculas_accion_1993 = movies[(movies['genres'].str.contains('Action', case=False, na=False)) & (movies['año'] == '1993')]

print(peliculas_accion_1993)


       movieId                                              title  \
281        284                   New York Cop (Nyû Yôku no koppu)   
430        434                                        Cliffhanger   
438        442                                     Demolition Man   
460        464                                        Hard Target   
461        465                                     Heaven & Earth   
...        ...                                                ...   
42832   169820                                     To Be the Best   
44036   172621  It's sunny on Deribassovskaya, or: It's rainin...   
44253   173117                         Quest of the Delta Knights   
44359   173359                                    Sharpe's Rifles   
45410   176191                                        Fit to Kill   

                                genres   año  
281                       Action|Crime  1993  
430          Action|Adventure|Thriller  1993  
438            Action|Adventur

### 1.8. Tratamiento columna de fecha UNIX

El campo `timestamp` de la tabla `ratings` está en formato UNIX, esto quiere decir que se trata de un número entero que representa los segundos que han pasado desde el 1 de enero de 1970. Para convertirlo a un formato más legible podemos utilizar la función `to_datetime` de Pandas. Crea una nueva columna en la tabla `ratings` llamada `date` que contenga la fecha en un formato legible. Cuando fue la primera y la última valoración?

In [147]:
# Convertir timestamp
ratings['date'] = pd.to_datetime(ratings['timestamp'], unit='s')

# Obtener la primera y la última valoración
primera_valoracion = ratings['date'].min()
ultima_valoracion = ratings['date'].max()

print(f'Primera valoración: {primera_valoracion}')
print(f'Última valoración: {ultima_valoracion}')


Primera valoración: 1996-03-27 17:20:36
Última valoración: 2017-08-04 04:52:46


## Ejericio 2: Estudiantes

En este ejercicio vamos a trabajar con un conjunto de datos que contiene información sobre estudiantes y sus notas. El fichero *students-mat.csv* ha sido extraído del repositorio de Machine Learning de la UCI. Para consultar la descripción completa del conjunto de datos, puedes visitar el siguiente enlace: [Student Performance Data Set](https://archive.ics.uci.edu/ml/datasets/Student+Performance).

In [149]:
import pandas as pd

### 2.1. Carga de Datos

Lee el fichero CSV llamado *students-mat.csv* y almacénalo en un DataFrame de Pandas llamado `students`. ¡OJO! El separador de campos es el punto y coma y no el comúnmente utilizado, la coma, que es el que se utiliza por defecto en la función `read_csv`. ¿Cómo puedes modificar el separador? Una vez cargado el fichero, muestra el número de filas y columnas del DataFrame.

In [151]:
students = pd.read_csv('student-mat.csv', sep=';')
print(f"Número de filas y columnas: {students.shape}")

Número de filas y columnas: (395, 33)


### 2.2. Filtro estudiantes mayores de edad

En nuestro estudio vamos a considerar solo a los estudiantes menores de 18 años, sobrescribe el DataFrame `students` para que contendrá solo a los estudiantes menores de edad.

In [177]:
students = students[students['age'] < 18]
print(students.head())

  school sex  age activities  grade1  grade2  grade3  grade1_passed  \
1     GP   F   17         no       5       5       6          False   
2     GP   F   15         no       7       8      10          False   
3     GP   F   15        yes      15      14      15           True   
4     GP   F   16         no       6      10      10          False   
5     GP   M   16        yes      15      15      15           True   

   grade2_passed  grade3_passed  all_passed  none_passed  final_grade  
1          False          False       False         True     5.333333  
2          False           True       False        False     8.333333  
3           True           True        True        False    14.666667  
4           True           True       False        False     8.666667  
5           True           True        True        False    15.000000  


### 2.3. Renombrar columnas

Las columnas G1, G2 y G3 contienen las notas de los estudiantes en tres cursos. Renombra estas columnas a `grade1`, `grade2` y `grade3` respectivamente. Para ello, puedes utilizar el método `rename` de Pandas.

In [173]:
students.rename(columns={'G1': 'grade1', 'G2': 'grade2', 'G3': 'grade3'}, inplace=True)
print(students.head())

  school sex  age activities  grade1  grade2  grade3  grade1_passed  \
1     GP   F   17         no       5       5       6          False   
2     GP   F   15         no       7       8      10          False   
3     GP   F   15        yes      15      14      15           True   
4     GP   F   16         no       6      10      10          False   
5     GP   M   16        yes      15      15      15           True   

   grade2_passed  grade3_passed  all_passed  none_passed  final_grade  
1          False          False       False         True     5.333333  
2          False           True       False        False     8.333333  
3           True           True        True        False    14.666667  
4           True           True       False        False     8.666667  
5           True           True        True        False    15.000000  


### 2.4. Filtrado de columnas

Vamos a reducir el tamaño del dataframe a solo las columnas que nos interesan. Mantén las siguientes columnas: `school`, `sex`, `age`, `activities` y las renombradas `grade1`, `grade2`, `grade3`. Sobrescribe el DataFrame `students` con el resultado.

In [157]:
students = students[['school', 'sex', 'age', 'activities', 'grade1', 'grade2', 'grade3']]

### 2.5. Calculo de aprobados por curso

Primero crea tres columnas de tipo booleano (`grade1_passed`, `grade2_passed`, `grade3_passed`) que indiquen si el estudiante ha aprobado cada uno de los cursos. Un estudiante aprueba si su nota es mayor o igual a 10. 

In [159]:
students['grade1_passed'] = students['grade1'] >= 10
students['grade2_passed'] = students['grade2'] >= 10
students['grade3_passed'] = students['grade3'] >= 10

Ahora puedes contar el número de estudiantes que han aprobado cada curso. ¿Cuántos estudiantes han aprobado los tres cursos? ¿Y cuántos no han aprobado ninguno?

In [183]:
# Contar aprobados en cada curso
grade1_passed_count = students['grade1_passed'].sum()
grade2_passed_count = students['grade2_passed'].sum()
grade3_passed_count = students['grade3_passed'].sum()
print(f"Aprobados en grade1: {grade1_passed_count}")
print(f"Aprobados en grade2: {grade2_passed_count}")
print(f"Aprobados en grade3: {grade3_passed_count}")

# Contar estudiantes que aprobaron los tres cursos y los que no aprobaron ninguno
students['all_passed'] = students[['grade1_passed', 'grade2_passed', 'grade3_passed']].all(axis=1)
students['none_passed'] = students[['grade1_passed', 'grade2_passed', 'grade3_passed']].sum(axis=1) == 0
all_passed_count = students['all_passed'].sum()
none_passed_count = students['none_passed'].sum()
print(f"Estudiantes que aprobaron los tres cursos: {all_passed_count}")
print(f"Estudiantes que no aprobaron ninguno: {none_passed_count}")

Aprobados en grade1: 188
Aprobados en grade2: 190
Aprobados en grade3: 204
Estudiantes que aprobaron los tres cursos: 167
Estudiantes que no aprobaron ninguno: 61


### 2.6. Nota media por estudiante

Calcula la nota media de cada estudiante y añade una nueva columna al DataFrame llamada `final_grade`.

In [185]:
students['final_grade'] = students[['grade1', 'grade2', 'grade3']].mean(axis=1)
print(students[['grade1', 'grade2', 'grade3', 'final_grade']].head())

   grade1  grade2  grade3  final_grade
1       5       5       6     5.333333
2       7       8      10     8.333333
3      15      14      15    14.666667
4       6      10      10     8.666667
5      15      15      15    15.000000


### 2.7. Nota media final por edad y sexo

Calcula la nota media final de los estudiantes por edad y sexo, redondea el resultado a dos decimeles. Almacena el resultado en un DataFrame llamado `final_grade_stats` y ordena el resultado por nota media de mayor a menor. 

In [187]:
final_grade_stats = students.groupby(['age', 'sex'])['final_grade'].mean().round(2).reset_index()
final_grade_stats.sort_values(by='final_grade', ascending=False, inplace=True)
print(final_grade_stats)

   age sex  final_grade
1   15   M        12.57
3   16   M        11.66
4   17   F        10.82
2   16   F        10.49
5   17   M        10.16
0   15   F         9.80


### 2.8. Valores duplicados

Hay alguna fila duplicada en el DataFrame `students`? ¿Crees que tiene sentido que haya filas duplicadas en este DataFrame? ¿Las eliminarías?

El método `drop_duplicates` de Pandas nos permite eliminar filas duplicadas pero ¿cómo sabemos si hay filas duplicadas? Puedes utilizar el método `duplicated` para comprobar si hay filas duplicadas en el DataFrame.

In [189]:
duplicated_rows = students.duplicated().sum()
print(f"Número de filas duplicadas: {duplicated_rows}")

Número de filas duplicadas: 15
